In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dropout,Dense,Embedding,Flatten
from tensorflow.keras.preprocessing.text import one_hot
import time
from tensorflow import make_ndarray

np.random.seed(42)

In [2]:
pip install torch._C

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch._C (from versions: none)
ERROR: No matching distribution found for torch._C
You should consider upgrading via the 'C:\Users\sur\PYTHON\python.exe -m pip install --upgrade pip' command.


In [3]:
chunk=pd.read_csv(r'F:\movie_set\rate.csv',chunksize=1000000)
df=pd.concat(chunk)

In [4]:
rand_userIds = np.random.choice(df['userId'].unique(), 
                                size=int(len(df['userId'].unique())*0.3), 
                                replace=False)

df = df.loc[df['userId'].isin(rand_userIds)]

In [5]:
df['rank_latest'] = df.groupby(['userId'])['timestamp'] \
                                .rank(method='first', ascending=False)

In [6]:
train_ratings = df[df['rank_latest'] != 1]
test_ratings = df[df['rank_latest'] == 1]

In [7]:
train_ratings

,userId,movieId,rating,timestamp,rank_latest
96,3,60,3.0,1298861675,44.0
97,3,110,4.0,1298922049,18.0
98,3,247,3.5,1298861637,47.0
99,3,267,3.0,1298861761,40.0
100,3,296,4.5,1298862418,33.0
...,...,...,...,...,...
99816,668,3000,3.0,993613196,18.0
99817,668,3039,3.0,993613196,19.0
99818,668,3213,3.0,993613359,15.0
99819,668,4012,3.0,993613196,20.0


In [8]:
train_ratings = train_ratings[['userId', 'movieId', 'rating']]
test_ratings = test_ratings[['userId', 'movieId', 'rating']]

In [9]:
train_ratings=train_ratings.astype({'userId':str,'movieId':str,'rating':float})

In [10]:
train_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30421 entries, 96 to 99820
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   30421 non-null  object 
 1   movieId  30421 non-null  object 
 2   rating   30421 non-null  float64
dtypes: float64(1), object(2)
memory usage: 950.7+ KB


In [11]:
train_ratings.loc[:, 'rating'] = 1


In [12]:
all_movieids = df['movieId'].unique().astype(str)
all_movieids

array(['60', '110', '247', ..., '137595', '138204', '6425'], dtype='<U21')

In [13]:
#class MovieLensTrainDataset(Dataset):
#     def __len__(self):
#        return len(self.users)
#  
#    def __getitem__(self, idx):
#        return self.users[idx], self.items[idx], self.labels[idx]#

def get_dataset( ratings, all_movieIds):
    user_vocab=220
    movie_vocab=6300
    
    data=[]
    labels=[]
    #users, items, labels = [], [], []
    user_item_set = set(zip(ratings['userId'], ratings['movieId']))
        
    num_negatives = 4
    for u, i in user_item_set:
        datapoint=[]
        datapoint.append(one_hot(u,user_vocab))
        datapoint.append(one_hot(i,movie_vocab))
        datapoint.append([1])
        data.append(datapoint)
        for _ in range(num_negatives):
            dpoint=[]
            negative_item = np.random.choice(all_movieIds)
            while (u, negative_item) in user_item_set:
                negative_item = np.random.choice(all_movieIds)
            dpoint.append(one_hot(u,user_vocab))
            dpoint.append(one_hot(negative_item,movie_vocab))
            dpoint.append([0])
            data.append(dpoint)
    return np.array(data)

In [14]:
data=get_dataset(train_ratings,all_movieids)
type(float(data[0][2][0]))

float

In [15]:
data[0]

array([[  57],
       [1313],
       [   1]])

In [16]:
train_dataset = tf.data.Dataset.from_tensor_slices(data).batch(64)

In [17]:
class NCF(Model):
    def __init__(self,user_vocab,movie_vocab):
        super().__init__()
        self.user_embedding=Embedding(user_vocab,8)
        self.movie_embedding=Embedding(movie_vocab,8)
        self.fc1=Dense(64,activation='relu')
        self.fc2=Dense(32,activation='relu')
        self.final=Dense(1,activation='sigmoid')
        self.fltn=Flatten()
    def forward(self, user,movie):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user)
        movie_embedded = self.movie_embedding(movie)

        # Concat the two embedding layers
        vector = tf.concat((user_embedded, movie_embedded), axis=-0)
        vector=self.fltn(vector)
  

        # Pass through dense layer
        vector = self.fc1(vector)
        vector = self.fc2(vector)

        # Output layer
        pred = self.final(vector)

        return pred
    

In [18]:
model=NCF(220,6300)


In [19]:
optimizer = tf.keras.optimizers.Adam(1e-4)

In [20]:
@tf.function
def train_step(batch,model):

    with tf.GradientTape() as tape:
        loss=float(0)
        for i in batch: 
            y_pred= model.forward(i[0][0],i[1][0])
            error=(float(i[2][0])-float(y_pred))**2
            for i in error:
                loss+=i[0]
            #print(" done")
        loss= loss/float(batch.shape[0]) 
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    

In [21]:
def train(dataset, model, epochs):
    for epoch in range(epochs):
        start = time.time()

    for batch in dataset:
        train_step(batch,model)
        #print("batch completed {}".format(time.time()))
        

    print ('Time_____ for_______ epoch______ {} is {} sec'.format(epoch + 1, time.time()-start))

  

In [22]:

train(train_dataset,model,1)



Time_____ for_______ epoch______ 1 is 114.2774658203125 sec


In [24]:
train_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30421 entries, 96 to 99820
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   userId   30421 non-null  object
 1   movieId  30421 non-null  object
 2   rating   30421 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 950.7+ KB


In [25]:
rand_user=np.random.choice(test_ratings['userId'],size=1)
indx=test_ratings['userId'].index[test_ratings['userId'].astype(int)==int(rand_user)][0]
indx

90729

In [26]:
test_movie=test_ratings['movieId'][indx]

In [27]:
combined=[]
for u,m in zip(train_ratings['userId'],train_ratings['movieId']):
    combined.append(str(u)+" "+str(m))
    

In [28]:
not_interacted=[]
counter=0
while counter<99:
    random_set=np.random.choice(combined,size=1)
    if int(random_set[0].split()[0])==int(rand_user):
        continue
    not_interacted.append(random_set[0].split()[1])
    counter+=1
    

     
        
        

In [29]:
not_interacted.append(test_movie)

In [30]:
total_100=not_interacted

In [31]:
len(total_100)

100

In [32]:
use=[]
for i in range(100):
    use.append(rand_user[0])

In [33]:
use

[602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602,
 602]

In [34]:
use=np.array([one_hot(str(i),220) for i in use])

In [35]:
mov=np.array([one_hot(str(i),6300) for i in total_100])

In [44]:
use[0][0]

18

In [41]:
use[0]

array([18])

In [56]:
pred=[]
#for u,m in zip(use,mov):
p=model.forward(int(use[0]),int(mov[0]))
pred.append([p,m[0]])

In [57]:
pred[0][0]

<tf.Tensor: shape=(16, 1), dtype=float32, numpy=
array([[0.18643588],
       [0.20347267],
       [0.18992358],
       [0.18367699],
       [0.21829745],
       [0.20064172],
       [0.17781907],
       [0.2101543 ],
       [0.32791054],
       [0.35752624],
       [0.25030988],
       [0.19722322],
       [0.19519106],
       [0.20455524],
       [0.3586344 ],
       [0.29480207]], dtype=float32)>

In [59]:
a=Embedding(220,8)(26)
b=Embedding(6300,8)(1712)

In [63]:
v=tf.concat((a,b),axis=0)
